In [ ]:
#| export callbacks

In [ ]:
#| include: false
%load_ext autoreload
%autoreload 2
from ipynb_path import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
from haiku_trainer.callbacks import *
import haiku as hk
import tqdm

In [ ]:
class ProgbarLogger(Callback):
    def __init__(self, ): pass

## Test

### Fake Module

In [ ]:
def make_hk_module(output_size: int = 2):
    """Creates a Haiku module with a linear layer and batchnorm."""
    def model(x, is_training=True):
        return hk.BatchNorm(True, True, 0.9)(
            hk.Linear(output_size)(x), is_training=is_training)
    
    return hk.transform_with_state(model)

In [ ]:
module = make_hk_module()

### Fake Data

In [ ]:
from jax_dataloader import DataLoader, ArrayDataset
from sklearn.datasets import make_classification

In [ ]:
xs, ys = make_classification(n_samples=2000, n_features=10, random_state=0)
ds = ArrayDataset(xs, ys)
dl = DataLoader(ds, 'jax', batch_size=128)

### Training

In [ ]:
trainer = Trainer(
    transformed=module,
    optimizers=optax.adam(1e-3),
    callbacks=[],
)

In [ ]:
trainer.fit(dl)

/home/birk/mambaforge-pypy3/envs/dev/lib/python3.9/site-packages/haiku/_src/base.py:515: UserWarning: Explicitly requested dtype float64 requested in zeros is not available, and will be truncated to dtype float32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  param = init(shape, dtype)
